# Team Unicorn

![](http://www.levycreative.com/wp/wp-content/uploads/2015/01/10945748_10152844879967949_5392963414676228818_n.jpg)
## Members
* Jackie Mcguire
* Todd Fox
* Jericho Herrera
* Nicki Lang

In [44]:
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *
import numpy
import pandas as pd
from yahoo_quote_download import yqd
import os
import datetime
from datetime import date as d
from datetime import timedelta
import quandl
import apikeys
from decimal import Decimal
import time as t
from dateutil.relativedelta import relativedelta

In [45]:
quandl.ApiConfig.api_key = apikeys.QUANDL_API

In [46]:
stocks = pd.read_csv('techiposince2001.csv')

In [47]:
stocks.dtypes

Ticker        object
Name          object
LastSale     float64
MarketCap    float64
ADR TSO       object
IPOyear       object
Sector        object
Industry      object
dtype: object

In [48]:
stocks = stocks.loc[stocks['ADR TSO'] == 'n/a']

In [49]:
stocks.MarketCap = stocks.MarketCap.apply(lambda x: round(x/10000))
stocks.MarketCap = stocks.MarketCap.apply(lambda y: '${:,.0f}'.format(y))

In [50]:
stocks=stocks.loc[stocks["IPOyear"]!='n/a']
stocks=stocks.loc[stocks["IPOyear"].astype(int)>=2000]
stocks.head()
stocks_sort = stocks.drop(['ADR TSO', 'LastSale', 'Sector'], axis=1)
stocks_sort.head()

,Ticker,Name,MarketCap,IPOyear,Industry
0,ACMR,"ACM Research, Inc.","$8,916",2017,Industrial Machinery/Components
1,ALTR,Altair Engineering Inc.,"$139,644",2017,Computer Software: Prepackaged Software
3,AYX,"Alteryx, Inc.","$158,743",2017,Computer Software: Prepackaged Software
4,APPN,Appian Corporation,"$125,190",2017,Computer Software: Prepackaged Software
5,AQ,Aquantia Corp.,"$36,264",2017,Semiconductors


In [51]:
def load_quote(ticker, startdate, enddate):
	print('===', ticker, '===')
	print(yqd.load_yahoo_quote(ticker, startdate, enddate))
	print(yqd.load_yahoo_quote(ticker, startdate, enddate, 'dividend'))
	print(yqd.load_yahoo_quote(ticker, startdate, enddate, 'split'))
def save_quote(ticker, startdate, enddate, filename):
    quotecsv = (yqd.load_yahoo_quote(ticker, startdate, enddate))
    with open(filename, 'a') as myfile:
        header=True
        for row in quotecsv:
            if header == True:
                myfile.write(row+','+'Ticker')
                myfile.write('\n')
                header=False
            else:
                myfile.write(row+','+ticker)
                myfile.write('\n')
            
def test():
	# Download quote for stocks
	load_quote(ticker, startdate, enddate)

def quandl_call(ticker, date):
    quandl.get_table('WIKI/PRICES', ticker=ticker, date=date)    

In [52]:
# for row, year in zip(stocks["Ticker"],stocks["IPOyear"]):
#     try:
#         save_quote(row, str(year)+'0101', str(year)+'1231', 'firstyeardata.csv')
#     except:
#         None


In [53]:
firstyear_df = pd.read_csv('firstyeardata.csv')
firstyear_df["Date"] = pd.to_datetime(firstyear_df["Date"], errors='coerce')
def format_epoch(dt):
    return (dt - datetime.datetime(1970,1,1)).total_seconds()
def format_date(sec):
    try:
        return datetime.datetime.fromtimestamp(sec).strftime('%D')
    except:
        return None
now = dt.datetime.now().date()
# oneyearout= now+timedelta(years=1)
firstyear_df["Date"] = firstyear_df["Date"].apply(format_epoch)



In [54]:
IPOdate_df=firstyear_df.loc[firstyear_df.groupby("Ticker")['Date'].idxmin()]
IPOdate_df["Date"] = IPOdate_df["Date"].apply(format_date)
IPOdate_df=IPOdate_df.dropna(how='all')
IPOdate_df = pd.merge(stocks_sort, IPOdate_df, on='Ticker', how='outer')
IPOdate_df=IPOdate_df.drop(['High', 'Low', 'Volume', 'Open', 'Adj Close'], axis=1)
IPOdate_df["Date"] = pd.to_datetime(IPOdate_df["Date"], infer_datetime_format=True, format='%Y%m%d',)
IPOdate_df=IPOdate_df.rename(columns={"Date":"IPOdate",
                                     "Close": "IPOclose"})
IPOdate_df["IPOclose"]= pd.to_numeric(IPOdate_df["IPOclose"], errors='coerce')



In [55]:
IPOdate_df['date1yr']=IPOdate_df["IPOdate"].apply(lambda x: x+pd.DateOffset(years=1, days=-1)+BDay(1))
IPOdate_df['date3yr']=IPOdate_df["IPOdate"].apply(lambda x: x+pd.DateOffset(years=3, days=-1)+BDay(1))
IPOdate_df['date5yr']=IPOdate_df["IPOdate"].apply(lambda x: x+pd.DateOffset(years=5, days=-1)+BDay(1))
# IPOdate_df=IPOdate_df.loc[IPOdate_df["1yr"]<= (now+tdt(years=1))]
IPOdate_df



,Ticker,Name,MarketCap,IPOyear,Industry,IPOdate,IPOclose,date1yr,date3yr,date5yr
0,ACMR,"ACM Research, Inc.","$8,916",2017,Industrial Machinery/Components,2017-10-25,10.110000,2018-10-25,2020-10-26,2022-10-25
1,ALTR,Altair Engineering Inc.,"$139,644",2017,Computer Software: Prepackaged Software,2017-10-25,13.230000,2018-10-25,2020-10-26,2022-10-25
2,AYX,"Alteryx, Inc.","$158,743",2017,Computer Software: Prepackaged Software,2017-03-23,15.500000,2018-03-23,2020-03-23,2022-03-23
3,APPN,Appian Corporation,"$125,190",2017,Computer Software: Prepackaged Software,2017-05-24,15.010000,2018-05-24,2020-05-25,2022-05-24
4,AQ,Aquantia Corp.,"$36,264",2017,Semiconductors,2017-01-02,0.220000,2018-01-02,2020-01-02,2022-01-03
5,BAND,Bandwidth Inc.,"$37,939",2017,Computer Software: Prepackaged Software,2017-10-25,5.390000,2018-10-25,2020-10-26,2022-10-25
6,BKI,"Black Knight, Inc.","$691,382",2017,Computer Software: Prepackaged Software,2017-01-02,0.035000,2018-01-02,2020-01-02,2022-01-03
7,CARG,"CarGurus, Inc.","$327,396",2017,EDP Services,2017-10-11,27.580000,2018-10-11,2020-10-12,2022-10-11
8,CARS,Cars.com Inc.,"$191,670",2017,EDP Services,2017-05-31,25.590000,2018-05-31,2020-06-01,2022-05-31
9,CISN,Cision Ltd.,"$143,483",2017,Computer Software: Prepackaged Software,2017-06-29,10.290000,2018-06-29,2020-06-29,2022-06-29


In [56]:
def save_quote1yr (ticker, startdate, enddate, filename):
    quotecsv = (yqd.load_yahoo_quote(ticker, startdate, enddate))
    with open(filename, 'a') as myfile:
        header=True
        for row in quotecsv:
            if header == True:
                myfile.write(row+','+'Ticker,Period')
                myfile.write('\n')
                header=False
            else:
                myfile.write(row+','+ticker+',1yr')
                myfile.write('\n')
def save_quote3yr (ticker, startdate, enddate, filename):
    quotecsv = (yqd.load_yahoo_quote(ticker, startdate, enddate))
    with open(filename, 'a') as myfile:
        header=True
        for row in quotecsv:
            if header == True:
                myfile.write(row+','+'Ticker,Period')
                myfile.write('\n')
                header=False
            else:
                myfile.write(row+','+ticker+',3yr')
                myfile.write('\n')
def save_quote5yr (ticker, startdate, enddate, filename):
    quotecsv = (yqd.load_yahoo_quote(ticker, startdate, enddate))
    with open(filename, 'a') as myfile:
        header=True
        for row in quotecsv:
            if header == True:
                myfile.write(row+','+'Ticker,Period')
                myfile.write('\n')
                header=False
            else:
                myfile.write(row+','+ticker+',5yr')
                myfile.write('\n')

In [57]:
for idx, row in IPOdate_df.iterrows():
    try:
        save_quote1yr(row['Ticker'], row["date1yr"].strftime(format='%Y%m%d'), row['date1yr'].strftime(format='%Y%m%d'), 'performance.csv')
        save_quote3yr(row['Ticker'], row["date3yr"].strftime(format='%Y%m%d'), row['date3yr'].strftime(format='%Y%m%d'), 'performance.csv')
        save_quote5yr(row['Ticker'], row["date5yr"].strftime(format='%Y%m%d'), row['date5yr'].strftime(format='%Y%m%d'), 'performance.csv')

    except:
        None

In [74]:
quotes_df = pd.read_csv('performance.csv', parse_dates=['Date'])
quotes_df["Close"]= pd.to_numeric(quotes_df["Close"], errors='coerce')
quotes_df = quotes_df.drop(['Open','High', 'Low', "Adj Close", "Volume"], axis=1)
quotes_df = quotes_df.dropna()
clean_quotes_df = quotes_df.loc[quotes_df["Date"]!='Date'].copy()
clean_quotes_df = clean_quotes_df.sort_values(by=['Ticker', 'Date'], ascending=True)
clean_quotes_df = clean_quotes_df.drop_duplicates()
clean_quotes_df["Date"] = pd.to_datetime(clean_quotes_df["Date"], infer_datetime_format=True, format='%Y%m%d',)
clean_quotes_df=clean_quotes_df.rename(columns={"Date":"IPOdate"
                                              })
# IPOdate_df
clean_pivot = clean_quotes_df.pivot(index='Ticker', columns='Period', values='Close')
clean_pivot
clean_pivot=clean_pivot.reset_index()
merged = pd.merge(clean_pivot, IPOdate_df, how='outer', on='Ticker')
merged["return1yr"]= .10
for idx, row in merged.iterrows():
    row["return1yr"] = (Decimal(row['1yr']) - Decimal(row['IPOclose']))/Decimal(row['IPOclose'])
merged=merged.dropna(subset=['1yr', '3yr', '5yr'], thresh=1)
merged

,Ticker,1yr,3yr,5yr,Name,MarketCap,IPOyear,Industry,IPOdate,IPOclose,date1yr,date3yr,date5yr,return1yr
0,AAOI,16.959999,21.010000,NaN,"Applied Optoelectronics, Inc.","$75,280",2013,Semiconductors,2013-09-25,9.960000,2014-09-25,2016-09-26,2018-09-25,0.1
1,ACIA,47.709999,NaN,NaN,"Acacia Communications, Inc.","$158,533",2016,Semiconductors,2016-05-12,30.950001,2017-05-12,2019-05-13,2021-05-12,0.1
2,ACLS,51.560001,27.760000,29.360001,"Axcelis Technologies, Inc.","$96,124",2000,Industrial Machinery/Components,2000-07-10,95.750000,2001-07-10,2003-07-10,2005-07-11,0.1
3,AER,21.200001,8.360000,10.190000,Aercap Holdings N.V.,"$826,423",2006,Diversified Commercial Services,2006-11-20,23.020000,2007-11-20,2009-11-20,2011-11-21,0.1
4,AIRG,9.630000,NaN,NaN,"Airgain, Inc.","$8,994",2016,Radio And Television Broadcasting And Communic...,2016-08-11,8.000000,2017-08-11,2019-08-12,2021-08-11,0.1
5,AL,24.240000,NaN,NaN,Air Lease Corporation,"$452,811",2011,Diversified Commercial Services,2011-04-18,27.950001,2012-04-18,2014-04-18,2016-04-18,0.1
6,ALLT,5.270000,4.050000,16.299999,Allot Communications Ltd.,"$18,511",2006,Computer Communications Equipment,2006-11-15,13.810000,2007-11-15,2009-11-16,2011-11-15,0.1
7,ALRM,23.860001,NaN,NaN,"Alarm.com Holdings, Inc.","$177,955",2015,Computer Software: Prepackaged Software,2015-06-25,16.879999,2016-06-27,2018-06-25,2020-06-25,0.1
8,AMBA,18.090000,57.169998,50.349998,"Ambarella, Inc.","$193,107",2012,Semiconductors,2012-10-09,6.060000,2013-10-09,2015-10-09,2017-10-09,0.1
9,AMBR,9.230000,7.350000,NaN,"Amber Road, Inc.","$19,865",2014,Computer Software: Prepackaged Software,2014-03-20,17.000000,2015-03-20,2017-03-20,2019-03-20,0.1


In [542]:
IPOdate_df

,Ticker,Name,MarketCap,IPOyear,Industry,IPOdate,Open,Close,Adj Close,date1yr,date3yr,date5yr
0,ACMR,"ACM Research, Inc.","$8,916",2017,Industrial Machinery/Components,2017-10-25,10.230000,10.110000,10.110000,2018-10-25,2020-10-26,2022-10-25
1,ALTR,Altair Engineering Inc.,"$139,644",2017,Computer Software: Prepackaged Software,2017-10-25,13.230000,13.230000,13.230000,2018-10-25,2020-10-26,2022-10-25
2,AYX,"Alteryx, Inc.","$158,743",2017,Computer Software: Prepackaged Software,2017-03-23,17.250000,15.500000,15.500000,2018-03-23,2020-03-23,2022-03-23
3,APPN,Appian Corporation,"$125,190",2017,Computer Software: Prepackaged Software,2017-05-24,15.050000,15.010000,15.010000,2018-05-24,2020-05-25,2022-05-24
4,AQ,Aquantia Corp.,"$36,264",2017,Semiconductors,2017-01-02,0.220000,0.220000,0.220000,2018-01-02,2020-01-02,2022-01-03
5,BAND,Bandwidth Inc.,"$37,939",2017,Computer Software: Prepackaged Software,2017-10-25,5.390000,5.390000,5.390000,2018-10-25,2020-10-26,2022-10-25
6,BKI,"Black Knight, Inc.","$691,382",2017,Computer Software: Prepackaged Software,2017-01-02,0.035000,0.035000,0.035000,2018-01-02,2020-01-02,2022-01-03
7,CARG,"CarGurus, Inc.","$327,396",2017,EDP Services,2017-10-11,29.000000,27.580000,27.580000,2018-10-11,2020-10-12,2022-10-11
8,CARS,Cars.com Inc.,"$191,670",2017,EDP Services,2017-05-31,25.590000,25.590000,25.590000,2018-05-31,2020-06-01,2022-05-31
9,CISN,Cision Ltd.,"$143,483",2017,Computer Software: Prepackaged Software,2017-06-29,10.050000,10.290000,10.290000,2018-06-29,2020-06-29,2022-06-29
